In [1]:
from customdataset import CustomDataset
from SSD import *
from train_step import train_step
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [2]:
PATH=os.getenv("HOME")+"/aiffel/ECG_data/physionet.org/files/circor-heart-sound/1.0.3/training_data"
file_list = os.listdir(PATH)
txt_list = [os.path.join(PATH, file) for file in file_list if file.endswith(".txt")]

# 환자 아이디를 훈련 데이터셋과 테스트 데이터셋으로 나눔
train_patient_txt, test_patient_txt = train_test_split(txt_list, test_size=0.9, random_state=42)

# 결과 출력
#print("Train Patient IDs:", train_patient_txt[:1])
#print("Test Patient IDs:", test_patient_txt[:1])

#path, txt_list, filter_params, target_size, th, resizing
dataset = CustomDataset(PATH, train_patient_txt, target_size=(300, 300), th=5, resizing=True)


def my_collate_fn(batch):
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])  # sample[0]은 화상 gt
        targets.append(torch.FloatTensor(sample[1]))  # sample[1]은 어노테이션 gt

    imgs = torch.stack(imgs, dim=0)
    return imgs, targets
BATCHSIZE = 8
train_dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=True, collate_fn=my_collate_fn)


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
model = build_model("Train", input_channels=1)

In [4]:
train_step(model, train_dataloader,epoch_num = 2, batchsize=BATCHSIZE, is_wandb=False, device=DEVICE)

Epoch : 1 / 2
Current Batch 0 / 131 | Cls Loss : 6.394, Loc Loss : 7.959, Total Loss : 14.354 | 10 iter time 0.1480: 
Current Batch 50 / 131 | Cls Loss : 3.196, Loc Loss : 4.636, Total Loss : 7.832 | 10 iter time 3.2312: 
Current Batch 100 / 131 | Cls Loss : 2.898, Loc Loss : 4.712, Total Loss : 7.610 | 10 iter time 3.1695: 
Epoch : 1 / 2 of Total Loss : Total Loss : 1010.949 | 1 epoch update time : 8.50s
-----------------------------------------------
Already './objectdetection_model' DIR path
Epoch : 2 / 2
Current Batch 0 / 131 | Cls Loss : 3.026, Loc Loss : 4.876, Total Loss : 7.902 | 10 iter time 0.0595: 
Current Batch 50 / 131 | Cls Loss : 3.004, Loc Loss : 4.916, Total Loss : 7.921 | 10 iter time 3.1870: 
Current Batch 100 / 131 | Cls Loss : 2.463, Loc Loss : 4.187, Total Loss : 6.650 | 10 iter time 3.1484: 
Epoch : 2 / 2 of Total Loss : Total Loss : 915.579 | 1 epoch update time : 8.32s
-----------------------------------------------
Already './objectdetection_model' DIR path
